In [1]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_columns', None) 

In [2]:
rollcall_votes = pd.read_csv('HSall_rollcalls.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
rollcall_drops = ['vote_result', 'vote_desc', 'vote_question', 'nominate_log_likelihood',
                  'nominate_spread_2', 'nominate_spread_1', 'nominate_mid_2',
                  'nominate_mid_1', 'dtl_desc']

In [4]:
rollcall_votes = rollcall_votes.drop(rollcall_drops, axis=1)

In [5]:
rollcall_votes['bill_id'] = rollcall_votes[
    'bill_number'] + '-' + rollcall_votes['congress'].astype(str)
rollcall_votes = rollcall_votes[rollcall_votes['congress'] > 92].reset_index(drop=True)

In [6]:
rollcall_votes['congress'].unique()

array([ 93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])

### Same Chamber Average Vote Margin 

In [265]:
def rolling_vote_pass_margins(df):
    
    
    df['bill_id_vote_number'] = df.groupby(['bill_id']).cumcount()
    
    
     
    
    
    def prev_votes(x):
        win = df.loc[:, 'bill_id_vote_number'].iloc[x.shape[0]-1].astype('int')
        win = max(win,0)
        return pd.Series(x).rolling(window=win).mean().iloc[-1]
     
    prev_yeas = df.loc[:, 'yea_count'].expanding().apply(prev_votes)
    prev_nays = df.loc[:, 'nay_count'].expanding().apply(prev_votes)
    
    prev_pass_margin = ((prev_yeas / prev_nays))-1
    df['bill_rolling_pass_margin'] = prev_pass_margin
    
    
    
    return df    
    

In [7]:
rollcall_votes = rollcall_votes.dropna(subset= ['bill_id']).reset_index(drop=True) 

### Opposite Chamber Average Vote Margin 

In [8]:
member_votes = pd.read_csv('HSall_votes.csv')

In [9]:
member_votes.head()

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,1,House,1,154,6,61.1
1,1,House,1,259,9,99.6
2,1,House,1,379,1,100.0
3,1,House,1,649,1,59.2
4,1,House,1,786,1,97.7


In [10]:
master_vote_df = pd.merge(
    member_votes, rollcall_votes, how='left', left_on=['congress', 'chamber', 'rollnumber'],
right_on=['congress', 'chamber', 'rollnumber'])

In [11]:
master_vote_df['session'].unique()

array([nan,  2.,  1.])

In [12]:
master_vote_df = master_vote_df.dropna(subset=['bill_number'])

In [13]:
master_vote_df.head()

,congress,chamber,rollnumber,icpsr,cast_code,prob,date,session,clerk_rollnumber,yea_count,nay_count,bill_number,bill_id
10684722,93,House,2,226,6,100.0,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684723,93,House,2,267,1,99.6,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684724,93,House,2,465,1,100.0,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684725,93,House,2,633,1,63.6,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684726,93,House,2,789,1,100.0,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93


In [14]:
members = pd.read_csv('all_legislators.csv')

In [277]:
#house_ids = pd.read_csv('house_ids.csv')
#senate_ids = pd.read_csv('senate_ids.csv')

#all_ids = house_ids.append(senate_ids)

In [17]:
all_ids = members[['icpsr_id', 'bioguide_id', 'thomas_id', 'party']]

In [18]:
all_ids

,icpsr_id,bioguide_id,thomas_id,party
0,29389.0,B000944,136.0,Democrat
1,39310.0,C000127,172.0,Democrat
2,15408.0,C000141,174.0,Democrat
3,15015.0,C000174,179.0,Democrat
4,40703.0,C001070,1828.0,Democrat
5,49300.0,F000062,1332.0,Democrat
6,40700.0,K000367,1826.0,Democrat
7,29373.0,M000639,791.0,Democrat
8,29147.0,S000033,1010.0,Independent
9,29732.0,S000770,1531.0,Democrat


In [26]:
all_ids.dropna(axis=0, subset=['icpsr_id'], inplace=True)

/home/william/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
all_ids.reset_index(drop=True, inplace=True)
all_ids.isnull().sum()

In [31]:
all_ids['icpsr_id'] = all_ids['icpsr_id'].astype(int) 

/home/william/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
master_vote_df.head()

,congress,chamber,rollnumber,icpsr,cast_code,prob,date,session,clerk_rollnumber,yea_count,nay_count,bill_number,bill_id
10684722,93,House,2,226,6,100.0,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684723,93,House,2,267,1,99.6,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684724,93,House,2,465,1,100.0,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684725,93,House,2,633,1,63.6,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93
10684726,93,House,2,789,1,100.0,1973-01-03,NaN,NaN,208.0,204.0,HRES6,HRES6-93


In [30]:
master_vote_df.dtypes

congress              int64
chamber              object
rollnumber            int64
icpsr                 int64
cast_code             int64
prob                float64
date                 object
session             float64
clerk_rollnumber    float64
yea_count           float64
nay_count           float64
bill_number          object
bill_id              object
dtype: object

In [56]:
all_votes = pd.merge(
    master_vote_df, all_ids, how='left', left_on=['icpsr'],
right_on=['icpsr_id'])  

In [67]:
all_votes.head()

,congress,chamber,rollnumber,cast_code,prob,date,session,clerk_rollnumber,yea_count,nay_count,bill_number,bill_id,bioguide_id,thomas_id,party
2628,93,House,8,1,90.7,1973-02-07,NaN,NaN,177.0,217.0,HR2107,HR2107-93,A000216,00027,Republican
2629,93,House,8,6,88.1,1973-02-07,NaN,NaN,177.0,217.0,HR2107,HR2107-93,A000222,00032,Democrat
2630,93,House,8,6,95.7,1973-02-07,NaN,NaN,177.0,217.0,HR2107,HR2107-93,B000178,00059,Democrat
2631,93,House,8,1,26.9,1973-02-07,NaN,NaN,177.0,217.0,HR2107,HR2107-93,B000371,00077,Democrat
2632,93,House,8,6,96.2,1973-02-07,NaN,NaN,177.0,217.0,HR2107,HR2107-93,B000550,00095,Democrat


In [64]:
all_votes = all_votes.drop(columns=['icpsr_id'])
all_votes = all_votes.drop(columns=['icpsr'])

In [58]:
all_votes['thomas_id'].fillna(0, inplace=True)

In [59]:
def thomas_cleaner(df):
    df['thomas_id'] = df['thomas_id'].astype(int)
    df['thomas_id'] = df['thomas_id'].astype(str)
    results_list = []
    for string in df['thomas_id']:
        string_len = len(string)
        preceeding_zeroes = 5 - string_len
        results_list.append(str((preceeding_zeroes * '0') + string))
    df['thomas_id'] = results_list
    return df    

In [60]:
all_votes = thomas_cleaner(all_votes) 

In [62]:
def bill_filter(vote_df):
    results_list = []
    for string in vote_df['bill_id']:
        if 'E' in string:
            results_list.append(1)
        elif 'P' in string:
            results_list.append(1)
        elif 'J' in string:
            results_list.append(1)
        else:
            results_list.append(0) 
    vote_df['check_col'] = results_list
    vote_df = vote_df[vote_df['check_col'] == 0]
    vote_df = vote_df.drop('check_col', axis=1)
    return vote_df

In [66]:
all_votes = bill_filter(all_votes) 

In [70]:
all_votes.isna().sum()

congress            0
chamber             0
rollnumber          0
cast_code           0
date                0
yea_count           0
nay_count           0
bill_number         0
bill_id             0
bioguide_id    129266
thomas_id           0
party          129266
dtype: int64

In [69]:
all_votes = all_votes.drop(['prob', 'session', 'clerk_rollnumber'], axis=1)

In [73]:
all_votes.tail(5)

,congress,chamber,rollnumber,cast_code,date,yea_count,nay_count,bill_number,bill_id,bioguide_id,thomas_id,party,vote_id
11944275,116,Senate,224,1,2019-07-23,97.0,2.0,HR1327,HR1327-116,F000062,01332,Democrat,HR1327-116-224
11944276,116,Senate,224,1,2019-07-23,97.0,2.0,HR1327,HR1327-116,M001111,01409,Democrat,HR1327-116-224
11944277,116,Senate,224,1,2019-07-23,97.0,2.0,HR1327,HR1327-116,C001035,01541,Republican,HR1327-116-224
11944278,116,Senate,224,1,2019-07-23,97.0,2.0,HR1327,HR1327-116,E000285,01542,Republican,HR1327-116-224
11944279,116,Senate,224,1,2019-07-23,97.0,2.0,HR1327,HR1327-116,S000320,01049,Republican,HR1327-116-224


In [72]:
all_votes['vote_id'] = all_votes['bill_id'] + '-' + all_votes['rollnumber'].astype(str) 

In [74]:
vote_dict = {0: 0, 1: 1, 2: 1,
             3: 1, 4: -1, 5: -1,
            6: -1, 7: 0, 8: 0, 9: 0} 

In [75]:
vote_list = all_votes['cast_code']

In [76]:
len(all_votes)

8928572

In [77]:
len(vote_list)

8928572

In [78]:
test = vote_list.replace(vote_dict)

In [79]:
len(test)

8928572

In [80]:
all_votes['vote'] = test

In [88]:
all_votes.head(4)

,congress,chamber,rollnumber,date,yea_count,nay_count,bill_number,bill_id,bioguide_id,thomas_id,party,vote_id,vote
2628,93,House,8,1973-02-07,177.0,217.0,HR2107,hr2107-93,A000216,00027,Republican,hr2107-93-8,1
2629,93,House,8,1973-02-07,177.0,217.0,HR2107,hr2107-93,A000222,00032,Democrat,hr2107-93-8,-1
2630,93,House,8,1973-02-07,177.0,217.0,HR2107,hr2107-93,B000178,00059,Democrat,hr2107-93-8,-1
2631,93,House,8,1973-02-07,177.0,217.0,HR2107,hr2107-93,B000371,00077,Democrat,hr2107-93-8,1


In [85]:
all_votes.drop(['cast_code'], axis=1, inplace=True)

In [87]:
all_votes['bill_id'] = all_votes['bill_id'].str.lower()
all_votes['vote_id'] = all_votes['vote_id'].str.lower() 

In [89]:
all_votes.to_csv('data_store/votes/all_member_votes.csv', index=False) 

In [303]:
roll_call_votes.to_csv('all_rollcall_votes.csv', index=False) 

In [90]:
def stratify_votes(df, first_congress, last_congress, entity):
    file_count = 0 
    for i in range(first_congress, last_congress+1):
        sub_frame = df[df['congress'] == i].reset_index(drop=True)
        sub_frame.to_csv(('data_store/votes/congress_' + str(
            i) + '_' + entity + '_votes.csv'), index=False)
        print (('Saving congress_' + str(i) + '_' + entity + '_votes.csv...'))
        file_count+=1
    print ('Complete.') 
    print (file_count, 'file(s) saved.') 

In [30]:
stratify_votes(roll_call_votes, 93, 116, 'rollcall') 

Saving congress_93_rollcall_votes.csv...
Saving congress_94_rollcall_votes.csv...
Saving congress_95_rollcall_votes.csv...
Saving congress_96_rollcall_votes.csv...
Saving congress_97_rollcall_votes.csv...
Saving congress_98_rollcall_votes.csv...
Saving congress_99_rollcall_votes.csv...
Saving congress_100_rollcall_votes.csv...
Saving congress_101_rollcall_votes.csv...
Saving congress_102_rollcall_votes.csv...
Saving congress_103_rollcall_votes.csv...
Saving congress_104_rollcall_votes.csv...
Saving congress_105_rollcall_votes.csv...
Saving congress_106_rollcall_votes.csv...
Saving congress_107_rollcall_votes.csv...
Saving congress_108_rollcall_votes.csv...
Saving congress_109_rollcall_votes.csv...
Saving congress_110_rollcall_votes.csv...
Saving congress_111_rollcall_votes.csv...
Saving congress_112_rollcall_votes.csv...
Saving congress_113_rollcall_votes.csv...
Saving congress_114_rollcall_votes.csv...
Saving congress_115_rollcall_votes.csv...
Saving congress_116_rollcall_votes.csv...

In [91]:
stratify_votes(all_votes, 93, 116, 'member') 

Saving congress_93_member_votes.csv...
Saving congress_94_member_votes.csv...
Saving congress_95_member_votes.csv...
Saving congress_96_member_votes.csv...
Saving congress_97_member_votes.csv...
Saving congress_98_member_votes.csv...
Saving congress_99_member_votes.csv...
Saving congress_100_member_votes.csv...
Saving congress_101_member_votes.csv...
Saving congress_102_member_votes.csv...
Saving congress_103_member_votes.csv...
Saving congress_104_member_votes.csv...
Saving congress_105_member_votes.csv...
Saving congress_106_member_votes.csv...
Saving congress_107_member_votes.csv...
Saving congress_108_member_votes.csv...
Saving congress_109_member_votes.csv...
Saving congress_110_member_votes.csv...
Saving congress_111_member_votes.csv...
Saving congress_112_member_votes.csv...
Saving congress_113_member_votes.csv...
Saving congress_114_member_votes.csv...
Saving congress_115_member_votes.csv...
Saving congress_116_member_votes.csv...
Complete.
24 file(s) saved.


In [3]:
test_df = pd.read_csv('HSall_rollcalls.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test_df 

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,1,House,1,1789-05-16,NaN,NaN,41,8,0.484,-0.872,0.229,-0.378,-13.144,HR2,NaN,NaN,NaN,"TO AMEND H.R. 2, (1 STAT. 24, APP. 7-4-1789), ..."
1,1,House,2,1789-05-22,NaN,NaN,36,1,0.256,0.967,-0.090,1.791,-1.900,NaN,NaN,NaN,NaN,TO PASS THE RESOLUTION THAT IT APPEARS TO THIS...
2,1,House,3,1789-06-22,NaN,NaN,30,18,-0.394,-0.454,-0.176,0.922,-22.048,HR8,NaN,NaN,NaN,"TO AMEND H.R. 8, (1 STAT. 28, APP. 7-27-1789),..."
3,1,House,4,1789-06-22,NaN,NaN,31,19,-0.194,-0.981,0.081,-0.327,-30.355,HR8,NaN,NaN,NaN,"TO AMEND H.R. 8, SO AS TO ELIMINATE THE PRESID..."
4,1,House,5,1789-06-24,NaN,NaN,29,22,-0.197,-0.266,-0.168,0.984,-22.221,HR8,NaN,NaN,NaN,TO PASS H.R. 8.
5,1,House,6,1789-07-01,NaN,NaN,31,19,0.597,-0.802,-0.065,-0.203,-27.996,HR5,NaN,NaN,NaN,TO RECEDE FROM DISAGREEMENT TO CERTAIN SENATE ...
6,1,House,7,1789-08-10,NaN,NaN,30,16,0.194,0.774,0.108,0.474,-17.227,HR19,NaN,NaN,NaN,"TO PASS H.R. 19, (1 STAT. 70, APP. 9-22-89), A..."
7,1,House,8,1789-08-12,NaN,NaN,28,23,0.994,0.112,0.017,0.493,-28.920,HR20,NaN,NaN,NaN,"TO AMEND H.R. 20, APP. 7-22-1790), A BILL TO P..."
8,1,House,9,1789-08-18,NaN,NaN,16,34,0.189,0.149,0.259,-1.275,-13.323,NaN,NaN,NaN,NaN,TO ORDER THE PREVIOUS QUESTION ON REFERRAL OF ...
9,1,House,10,1789-08-21,NaN,NaN,17,32,0.108,0.061,0.327,-1.784,-10.011,NaN,NaN,NaN,NaN,TO AMEND THE CONSTITUTIONAL AMENDMENTS RESOLUT...
